In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
from sqlalchemy import create_engine

In [4]:
PG_HOST = '172.17.0.2'
pg_engine = create_engine(f'postgresql+psycopg2://postgres:password@{PG_HOST}/habr', pool_recycle=3600)
pg_conn = pg_engine.connect()

In [5]:
import pandas as pd

posts_df = pd.read_sql("select * from posts", pg_conn)
print("Posts count: ", len(posts_df))
posts_df.head(2)

Posts count:  232127


,post_id,title,text,date,views_count,comments_count,bookmarks_count,rating,author_nickname
0,365293,LinkedIn автоматизировал добавление в профиль ...,Разработчики LinkedIn объявили о появившейся в...,2015-03-26 16:32:00,7300,1,10,+8,Teachbase
1,5005,MobileFaker: фальшивые звонки как социальное о...,Со своего мобильника вы заказываете фальшивый ...,2007-01-26 14:39:00,315,17,2,+16,alizar


In [6]:
tags_df = pd.read_sql("select * from tags", pg_conn)
print("tags_df len: ", len(tags_df))
tags_df.head(2)

tags_df len:  1054820


,post_id,tag
0,365293,LinkedIn
1,365293,онлайн-курсы


In [7]:
habs_df = pd.read_sql("select * from habs", pg_conn)
print("tags_df len: ", len(habs_df))
habs_df.head(2)

tags_df len:  447893


,post_id,hab
0,365293,Блог компании Teachbase.ru
1,365293,Социальные сети и сообщества


In [8]:
habs_df['hab'].value_counts()

Чулан                                 29221
Разработка веб-сайтов                 12309
Программирование                      11680
Информационная безопасность           11232
IT-компании                            8710
                                      ...  
Блог компании Дараба                      1
Блог компании McKinsey & Company          1
Блог компании Grand-Hosting.ru            1
Блог компании Две говорящие головы        1
Блог компании Sibnet.ru                   1
Name: hab, Length: 2269, dtype: int64

In [9]:
tags_df['tag'].value_counts()

javascript              4473
android                 4011
google                  3726
linux                   3274
php                     3224
                        ... 
productfest                1
Obertur Technologies       1
бокэ                       1
пленочная slr              1
http codes                 1
Name: tag, Length: 204532, dtype: int64

In [10]:
def get_habs_list(post_id: int) -> str:
    habs = ''
    for _, row in habs_df[habs_df['post_id'] == post_id].iterrows():
        habs += (row['hab'] + '<SEP>')
    return habs[:-5]

get_habs_list(5005)

'Чулан'

In [11]:
def get_tags_list(post_id: int) -> str:
    tags = ''
    for _, row in tags_df[tags_df['post_id'] == post_id].iterrows():
        tags += (row['tag'] + '<SEP>')
    return tags[:-5]

get_tags_list(5005)

'MobileFaker<SEP>фальшивые звонки<SEP>лицемерие<SEP>мобильные услуги<SEP>соци'

In [12]:
import tqdm

posts_df['habs'] = posts_df.apply(lambda x: get_habs_list(x['post_id']), axis=1)
posts_df['tags'] = posts_df.apply(lambda x: get_tags_list(x['post_id']), axis=1)
posts_df.head(2)

,post_id,title,text,date,views_count,comments_count,bookmarks_count,rating,author_nickname,habs,tags
0,365293,LinkedIn автоматизировал добавление в профиль ...,Разработчики LinkedIn объявили о появившейся в...,2015-03-26 16:32:00,7300,1,10,+8,Teachbase,Блог компании Teachbase.ru<SEP>Социальные сети...,LinkedIn<SEP>онлайн-курсы<SEP>сертификаты<SEP>...
1,5005,MobileFaker: фальшивые звонки как социальное о...,Со своего мобильника вы заказываете фальшивый ...,2007-01-26 14:39:00,315,17,2,+16,alizar,Чулан,MobileFaker<SEP>фальшивые звонки<SEP>лицемерие...


In [13]:
habs_dict = dict(habs_df['hab'].value_counts())
habs_list = list(habs_dict.keys())
len(habs_dict), list(habs_dict.items())[:1]

(2269, [('Чулан', 29221)])

In [14]:
tags_dict = dict(tags_df['tag'].value_counts())
tags_list = list(tags_dict.keys())
len(tags_dict), list(tags_dict.items())[:1]

(204532, [('javascript', 4473)])

In [15]:
class Habs2Vec:
    
    habs2code: dict
    code2hab: dict
    
    def fit(self, habs_list: list) -> None:
        self.habs2code = {hab: i for i, hab in enumerate(habs_list)}
        self.code2hab = {i: hab for i, hab in enumerate(habs_list)}
        
    def transform(self, post_habs: list) -> np.array:
        vec = np.zeros(len(self.habs2code), dtype=np.int0)
        for hab in post_habs:
            vec[self.habs2code[hab]] = 1
        return vec
    
    def encode_vec(self, vec: np.array) -> list:
        habs = []
        for i, elem in enumerate(vec):
            if elem:
                habs.append(self.code2hab[i])
        return habs
    
transformer = Habs2Vec()
transformer.fit(habs_list)
print('Habs: ', habs_list[:2] + habs_list[-2:])
vec = transformer.transform(habs_list[:2] + habs_list[-2:])
print('Binary vector: ', vec)
print('Decoded binary vector: ', transformer.encode_vec(vec))

Habs:  ['Чулан', 'Разработка веб-сайтов', 'Блог компании Две говорящие головы', 'Блог компании Sibnet.ru']
Binary vector:  [1 1 0 ... 0 1 1]
Decoded binary vector:  ['Чулан', 'Разработка веб-сайтов', 'Блог компании Две говорящие головы', 'Блог компании Sibnet.ru']


In [16]:
decoded_habs_list = [transformer.transform(post['habs'].split('<SEP>')) for _, post in posts_df.iterrows()]
len(decoded_habs_list), decoded_habs_list[:2]

(232127, [array([0, 0, 0, ..., 0, 0, 0]), array([1, 0, 0, ..., 0, 0, 0])])

In [40]:
habs_stat = {}
tags_stat = {}
authors_stat = {}

def parse_rating(rating: str) -> int:
    if rating[0] == '–':
        return -int(rating[1:])
    return int(rating)

for hab in habs_list:
    habs_stat[hab] = {
        'comments_count': 0,
        'rating': 0,
        'bookmarks_count': 0,
        'views_count': 0,
        'authors_set': set(),
        'count': 0
    }
    for _, hab_row in habs_df[habs_df['hab']==hab].iterrows():
        post = posts_df[posts_df['post_id']==hab_row['post_id']].iloc[0]
        habs_stat[hab]['comments_count'] += post['comments_count']
        habs_stat[hab]['bookmarks_count'] += post['bookmarks_count']
        habs_stat[hab]['views_count'] += post['views_count']
        habs_stat[hab]['authors_set'].add(str(post['author_nickname']))
        habs_stat[hab]['rating'] += parse_rating(str(post['rating']).strip())
        habs_stat[hab]['count'] += 1
        
list(habs_stat.items())[:1]

[('Чулан',
  {'comments_count': 605766,
   'rating': 205596,
   'bookmarks_count': 137265,
   'views_count': 20280127,
   'authors_set': {'DKurilo',
    'DickRishellier',
    'SergeyGrigorev',
    'boatincow',
    'revazov',
    'Adam_B',
    'Orenlab',
    'dizelbox',
    'kattt',
    'melange',
    'Rand0M',
    'maroonorg',
    'ymik',
    'GAZone',
    'bor1s',
    'retvizan',
    'chibis',
    'Quimby',
    'mega_winner',
    'jitscar',
    'E_Z',
    'siboos',
    'EcoDark',
    'fjolner',
    'predtech',
    'yukey',
    'ixStream',
    'CINick',
    'phasma',
    'Sigura',
    'gene4000',
    'tasman',
    'mihteh',
    'Zert',
    'c0mpaz',
    'glebis',
    'vsh',
    'iderins',
    'alexchin',
    'Ango',
    'levide',
    'asoloviev',
    'sergray',
    'arvie',
    'zhekanax',
    'romanser',
    'dsamsonov',
    'statuslab',
    'mystafa',
    'jasiejames',
    'Tyran',
    'SLY_G',
    'iloveyoutoo',
    'karazyabko',
    'Stepanow',
    'antongridz',
    'samokhvalov',


In [41]:
for tag in tags_list:
    tags_stat[tag] = {
        'comments_count': 0,
        'rating': 0,
        'bookmarks_count': 0,
        'views_count': 0,
        'authors_set': set(),
        'count': 0
    }
    for _, tag_row in tags_df[tags_df['tag']==tag].iterrows():
        post = posts_df[posts_df['post_id']==tag_row['post_id']].iloc[0]
        tags_stat[tag]['comments_count'] += post['comments_count']
        tags_stat[tag]['bookmarks_count'] += post['bookmarks_count']
        tags_stat[tag]['views_count'] += post['views_count']
        tags_stat[tag]['authors_set'].add(str(post['author_nickname']))
        tags_stat[tag]['rating'] += parse_rating(str(post['rating']).strip())
        
list(tags_stat.items())[:1]

NameError: name 'tags_stat' is not defined

In [ ]:
from sklearn.manifold import TSNE

X = decoded_habs_list
X_embedded = TSNE(n_components=2).fit_transform(X)